## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-26-03-51-34 +0000,nypost,Ex-Avis employee wanted for allegedly stealing...,https://nypost.com/2025/10/25/us-news/ex-avis-...
1,2025-10-26-03-48-28 +0000,nyt,Trump Announces Tariff Increase on Canada Over...,https://www.nytimes.com/2025/10/25/world/canad...
2,2025-10-26-03-29-31 +0000,startribune,Takeaways: Gophers embarrassed by Iowa in 41-3...,https://www.startribune.com/takeaways-minnesot...
3,2025-10-26-03-23-19 +0000,nyt,Honduran Man Struck and Killed On a Virginia H...,https://www.nytimes.com/2025/10/25/us/ice-hond...
4,2025-10-26-03-21-26 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/10/25/us/tru...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2304/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,36
87,he,13
76,asia,11
233,new,10
301,nyc,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
160,2025-10-25-09-38-00 +0000,wsj,Trump Pitches Meeting With Kim Jong Un During ...,https://www.wsj.com/world/asia/trump-pitches-m...,89
114,2025-10-25-14-56-21 +0000,nypost,Trump ‘open’ to meeting with Kim Jong Un durin...,https://nypost.com/2025/10/25/world-news/trump...,86
163,2025-10-25-09-10-25 +0000,bbc,Trump says he wants to meet Kim Jong Un on Asi...,https://www.bbc.com/news/articles/cn09gg8v721o...,77
16,2025-10-26-01-51-00 +0000,wsj,Trump Says He Will Raise Tariffs on Canada by ...,https://www.wsj.com/politics/policy/trump-hike...,74
24,2025-10-26-01-00-00 +0000,wsj,"“You have zero zoning conditions, you’re the p...",https://www.wsj.com/politics/policy/trump-whit...,71


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
160,89,2025-10-25-09-38-00 +0000,wsj,Trump Pitches Meeting With Kim Jong Un During ...,https://www.wsj.com/world/asia/trump-pitches-m...
37,44,2025-10-25-23-10-24 +0000,nypost,NYC voters run to the polls on first day of ea...,https://nypost.com/2025/10/25/us-news/nyc-vote...
175,39,2025-10-25-08-00-00 +0000,cbc,Palestinian man says he wishes he could see fa...,https://www.cbc.ca/news/world/gaza-palestinian...
24,35,2025-10-26-01-00-00 +0000,wsj,"“You have zero zoning conditions, you’re the p...",https://www.wsj.com/politics/policy/trump-whit...
113,31,2025-10-25-15-21-49 +0000,nypost,Chilling footage shows Russian drone hunting d...,https://nypost.com/2025/10/25/world-news/chill...
54,30,2025-10-25-21-58-46 +0000,nypost,Louvre heist being investigated as inside job ...,https://nypost.com/2025/10/25/world-news/louvr...
6,28,2025-10-26-03-08-19 +0000,nypost,Sister of slain NJ man blasts cop who allegedl...,https://nypost.com/2025/10/25/us-news/sister-o...
78,28,2025-10-25-18-55-14 +0000,nyt,Canada-U.S. Tensions Stay on Baseball Field Du...,https://www.nytimes.com/2025/10/25/sports/base...
147,26,2025-10-25-10-25-15 +0000,bbc,Who is new deputy Labour leader Lucy Powell?,https://www.bbc.com/news/articles/cev1d30kjdlo...
126,26,2025-10-25-13-07-30 +0000,bbc,Trump hopes China will help push Russia toward...,https://www.bbc.com/news/articles/cn51yk9vgrko...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
